# imports

In [4]:
import os
import cv2
import json
import zipfile
import shutil
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from sklearn.model_selection import train_test_split
from ultralytics import YOLO  # For detection model
from pycocotools.coco import COCO  # For COCO annotations

import torch
import tensorflow as tf

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# data reading

In [5]:


d1_path = '/content/drive/MyDrive/Auto-Assess/data sets/d1'
d2_path = '/content/drive/MyDrive/Auto-Assess/data sets/d2'

print(f"Contents of {d1_path}:")
try:
    for item in os.listdir(d1_path):
        print(item)
except FileNotFoundError:
    print(f"Error: Directory not found at {d1_path}")

print(f"\nContents of {d2_path}:")
try:
    for item in os.listdir(d2_path):
        print(item)
except FileNotFoundError:
    print(f"Error: Directory not found at {d2_path}")

Contents of /content/drive/MyDrive/Auto-Assess/data sets/d1:
train
img
val
test

Contents of /content/drive/MyDrive/Auto-Assess/data sets/d2:
validation
training


In [6]:
#merging data
import shutil

# Paths for your datasets on Google Drive
d1_path = '/content/drive/MyDrive/Auto-Assess/data sets/d1'
d2_path = '/content/drive/MyDrive/Auto-Assess/data sets/d2'

# Define target directories for the merged dataset structure
merged_base_dir = "merged_dataset"
merged_train_dir = os.path.join(merged_base_dir, "train")
merged_val_dir = os.path.join(merged_base_dir, "val")

# Create merged directories
os.makedirs(merged_train_dir, exist_ok=True)
os.makedirs(merged_val_dir, exist_ok=True)

print(f"Merging data from {d1_path} and {d2_path}...")

# Function to copy files, avoiding duplicates
def copy_files(source_dir, target_dir):
    if not os.path.exists(source_dir):
        print(f"Warning: Source directory not found: {source_dir}")
        return

    for root, _, files in os.walk(source_dir):
        relative_path = os.path.relpath(root, source_dir)
        target_root = os.path.join(target_dir, relative_path)
        os.makedirs(target_root, exist_ok=True)

        for file_name in files:
            source_file = os.path.join(root, file_name)
            target_file = os.path.join(target_root, file_name)

            if not os.path.exists(target_file):  # Avoid overwriting
                shutil.copy2(source_file, target_file)
            # else:
            #     print(f"Skipping existing file: {target_file}")


# Merge data from d1
copy_files(os.path.join(d1_path, 'train'), merged_train_dir)
copy_files(os.path.join(d1_path, 'val'), merged_val_dir)
# Assuming d1/img also contains training/validation images, merge them into train
copy_files(os.path.join(d1_path, 'img'), merged_train_dir)


# Merge data from d2 (mapping training to merged_train and validation to merged_val)
copy_files(os.path.join(d2_path, 'training'), merged_train_dir)
copy_files(os.path.join(d2_path, 'validation'), merged_val_dir)



Merging data from /content/drive/MyDrive/Auto-Assess/data sets/d1 and /content/drive/MyDrive/Auto-Assess/data sets/d2...


# preprocessing